# Import regions
This notebook creates a table for holding polygon shapes. These shapes can then be used to generate statistics in regions specified by those polygons.

## Prerequisites
This notebook assumes that you have already run the `import_socat` notebook.

## Setup
Imports, constants etc.

In [5]:
import sqlite3
from osgeo import ogr

DB_FILE = 'socat_kpi.sqlite'

## Connect to database and Clean
Connect to the database and delete any existing tables.

In [6]:
conn = sqlite3.connect(DB_FILE)
conn.enable_load_extension(True)
conn.load_extension("mod_spatialite")

try:
    conn.execute("SELECT DisableSpatialIndex('shapes', 'shape')")
except Exception as e:
    pass
    
conn.execute("DROP TABLE IF EXISTS shapes")
conn.commit()

## Create the Regions table
This table will contain the following fields:

| Field    | Description                        |
|----------|------------------------------------|
| `source` | The entity that defined the region |
| `name`   | The name of the region             |
| `shape`  | The polygon defining the region    |

Each record in the table will contain a single polygon.

In [7]:
conn.execute("""CREATE TABLE shapes(
id integer primary key,
source varchar(100),
name varchar(100),
shape MULTIPOLYGON
);""")

conn.execute('SELECT RecoverGeometryColumn("shapes", "shape", 4326, "MULTIPOLYGON", "XY")')

conn.commit()

## Import single-polygon files
This cell can be used to import files containing single polygons.

Specify the list of files in `POLY_FILES`, which is an array of arrays. Each sub-array should contain `[filename, source, name]`.

The import logic is adapted from [Importing shapefile to PostGIS using Python and ogr](https://gis.stackexchange.com/questions/90085/importing-shapefile-to-postgis-using-python-and-ogr).

In [9]:
POLY_FILES = [
    ['shapes/ospar_regions/ArcticWaters.shp', 'OSPAR', 'Arctic Waters'],
    ['shapes/ospar_regions/GreaterNorthSea.shp', 'OSPAR', 'Greater North Sea'],
    ['shapes/ospar_regions/CelticSeas.shp', 'OSPAR', 'Celtic Seas'],
    ['shapes/ospar_regions/BiscayIberia.shp', 'OSPAR', 'Bay of Biscay and Iberian Coast'],
    ['shapes/ospar_regions/WiderAtlantic.shp', 'OSPAR', 'Wider Atlantic'],
    ['shapes/my_regions/Baltic.shp', 'N/A', 'Baltic'],
    ['shapes/my_regions/Mediterranean.shp', 'N/A', 'Mediterranean']
]

for p in POLY_FILES:
    shapefile = ogr.Open(p[0])    
    layer = shapefile.GetLayer(0)    
    feature = layer.GetFeature(0)
    wkt = feature.GetGeometryRef().ExportToWkt()
    conn.execute(f"INSERT INTO shapes (source, name, shape) VALUES ('{p[1]}', '{p[2]}', CastToMultiPolygon(ST_GeometryFromText('{wkt}', 4326)))")
    conn.commit()


## CCAMLR Regions
Another example of regions around the Southern Ocean. One of the entries in this shapefile is a MULTIPOLYGON. For now I'm just ignoring it.

In [10]:
shapefile = ogr.Open('shapes/CCAMLR/asd/CCAMLR_ASD_EPSG4326.shp')
layer = shapefile.GetLayer(0)

feature = layer.GetNextFeature()
while feature is not None:
    geometry = feature.GetGeometryRef()
    if geometry.GetGeometryType() == ogr.wkbPolygon:
        wkt = feature.GetGeometryRef().ExportToWkt()
        conn.execute(f"INSERT INTO shapes (source, name, shape) VALUES ('CCAMLR', '{feature.GAR_Name}', CastToMultiPolygon(ST_GeometryFromText('{wkt}', 4326)))")
    feature = layer.GetNextFeature()
conn.commit()


## Create Index
Speed is king

In [11]:
conn.execute('SELECT CreateSpatialIndex("shapes", "shape")')
conn.commit()

updateTableTriggers: "table "idx_shapes_shape" already exists"


## Close everything down

In [7]:
conn.close()